In [1]:
from lcstack.core.parsers import parse_data_with_type, NamedMappingParserArgs
from lcstack.core.parsers import OutputParserType

mapping_pass_through = {
    None: NamedMappingParserArgs(
        name="foo",
        output_type=OutputParserType.pass_through
    )
}

mapping_message = {
    "bar": NamedMappingParserArgs(
        name="foo",
        output_type=OutputParserType.message,
        message_role = "ai"
    )
}

mapping_messages = {
    "bar": NamedMappingParserArgs(
        name="foo",
        output_type=OutputParserType.messages
    )
}

data = {"foo": "bar"}
print(parse_data_with_type(data=data, output_type=OutputParserType.struct, struct_mapping=mapping_pass_through))
print(parse_data_with_type(data=data, output_type=OutputParserType.struct, struct_mapping=mapping_message))
print(parse_data_with_type(data=data, output_type=OutputParserType.struct, struct_mapping=mapping_messages))

data_with_tool_calls = {"foo": {"tool_calls": [{"name": "tool_name", "args": {"foo": "bar"}}, {"name": "tool_2_name", "args": {"foo": "foo_bar"}}]}}
print(parse_data_with_type(data=data_with_tool_calls, output_type=OutputParserType.struct, struct_mapping=mapping_message))

USER_AGENT environment variable not set, consider setting it to identify your requests.


bar
{'bar': AIMessage(content='bar', additional_kwargs={}, response_metadata={})}
{'bar': [AIMessage(content='bar', additional_kwargs={}, response_metadata={})]}
{'bar': AIMessage(content='', additional_kwargs={}, response_metadata={}, tool_calls=[{'name': 'tool_name', 'args': {'foo': 'bar'}, 'id': 'tc_3537229e-b6e1-44c5-8d69-bc2de93156b9_0', 'type': 'tool_call'}, {'name': 'tool_2_name', 'args': {'foo': 'foo_bar'}, 'id': 'tc_3537229e-b6e1-44c5-8d69-bc2de93156b9_1', 'type': 'tool_call'}])}


### Langchain Output Parsers

In [1]:
import lcstack

lcstack.set_config_root("configs")

# type: "str", "json", "comma_separated_list", "markdown_list", "numbered_list", "json_tools", "pydantic_tools".

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
str_output_parser = """
str_parser:
  initializer: lc_output_parser
  data:
    type: str
    name: foo
"""

lcs = lcstack.LcStackBuilder.from_object(str_output_parser).build()
str_parser = lcs.get("str_parser")

print(str_parser)

from langchain_core.messages import AIMessage
msg = AIMessage(content="foo")
print(str_parser.invoke(msg))

name='foo'
foo


In [5]:
list_output_parser = """
list_parser:
  initializer: lc_output_parser
  data:
    type: comma_separated_list
    name: foo
"""

lcs = lcstack.LcStackBuilder.from_object(list_output_parser).build()
list_parser = lcs.get("list_parser")

print(list_parser)

from langchain_core.messages import AIMessage
msg = AIMessage(content="foo,bar,baz")
print(list_parser.invoke(msg))

name='foo'
['foo', 'bar', 'baz']


In [7]:
list_output_parser = """
list_parser:
  initializer: lc_output_parser
  data:
    type: markdown_list
    name: foo
"""

lcs = lcstack.LcStackBuilder.from_object(list_output_parser).build()
list_parser = lcs.get("list_parser")

print(list_parser)

from langchain_core.messages import AIMessage
# markdown_list
content = """
- foo
- bar
- baz
  - qux
  - quux
"""
msg = AIMessage(content=content)
print(list_parser.invoke(msg))

name='foo'
['foo', 'bar', 'baz', 'qux', 'quux']


In [9]:
list_output_parser = """
list_parser:
  initializer: lc_output_parser
  data:
    type: numbered_list
    name: foo
"""

lcs = lcstack.LcStackBuilder.from_object(list_output_parser).build()
list_parser = lcs.get("list_parser")

print(list_parser)

from langchain_core.messages import AIMessage
# numbered_list
content = """
1. foo
    1. bar
2. bar
3. baz
  1. qux
  2. quux
"""
msg = AIMessage(content=content)
print(list_parser.invoke(msg))

name='foo'
['foo', 'bar', 'bar', 'baz', 'qux', 'quux']


In [16]:
json_output_parser = """
json_parser:
  initializer: lc_output_parser
  data:
    type: json
    name: foo
    partial: true
"""

lcs = lcstack.LcStackBuilder.from_object(json_output_parser).build()
json_parser = lcs.get("json_parser")

print(json_parser)

from langchain_core.messages import AIMessage
# markdown json
content_1 = """
```json
{
  "foo": {
    "bar": "baz"
  }
}
```
"""
# plain json
content_2 = """
{
  "foo": {
    "bar": "baz"
  }
}
"""
# partial json
content_3 = """
{
  "foo": {
    "bar": "baz"
  },
  "foo2": {
    "bar2": "baz2
}

"""
msg = AIMessage(content=content_1)
print(json_parser.invoke(msg))


msg = AIMessage(content=content_2)
print(json_parser.invoke(msg))


msg = AIMessage(content=content_3)
print(json_parser.invoke(msg))

name='foo'
{'foo': {'bar': 'baz'}}
{'foo': {'bar': 'baz'}}
{'foo': {'bar': 'baz'}, 'foo2': {'bar2': 'baz2\n}'}}


In [20]:
tool_output_parser = """
tool_parser:
  initializer: lc_output_parser
  data:
    type: "json_tools" # "pydantic_tools".
    name: foo
    partial: true
"""

lcs = lcstack.LcStackBuilder.from_object(tool_output_parser).build()
tool_parser = lcs.get("tool_parser")

print(tool_parser)

from langchain_core.messages import AIMessage
# markdown json
content_1 = [
  {
    "name": "foo",
    "args": {"a": 1},
    "id": "123"
  }
]

msg = AIMessage(content="", tool_calls=content_1)
print(tool_parser.invoke(msg))

name='foo'
[{'args': {'a': 1}, 'type': 'foo'}]


In [7]:
tool_output_parser = """
tool_parser:
  initializer: lc_output_parser
  data:
    type: "pydantic_tools"
    name: foo
    tools: []
"""


from pydantic import BaseModel
from langchain_core.tools import tool


@tool
def foo(a: int):
    """Demo Tool"""
    return a

class Foo(BaseModel):
    """Demo Foo Model"""

    a: int
    """an int field"""

tool_output_parser = {
    "tool_parser": {
        "initializer": "lc_output_parser",
        "data": {
            "type": "pydantic_tools",
            "name": "foo",
            "tools": [Foo] # [Foo, foo()] # [Foo, foo]
        }
    }
}


lcs = lcstack.LcStackBuilder.from_object(tool_output_parser).build()
tool_parser = lcs.get("tool_parser")

print(tool_parser)

from langchain_core.messages import AIMessage
# markdown json
content_1 = [
  {
    "name": "Foo",
    "args": {"a": 1},
    "id": "123"
  },
]

msg = AIMessage(content="", tool_calls=content_1)
print(tool_parser.invoke(msg))

name='foo' tools=[<class '__main__.Foo'>]
[Foo(a=1)]


In [ ]:
from langchain_core.tools.convert import Convert